In [1]:
import pandas as pd

In [2]:
df_weather = pd.read_csv('./data/openweather/weather_data_summary.csv')
df_weather.head()

,date,precipitation,max_temp,min_temp,temperature,pressure,humidity
0,2010-12-31,5.95,301.39,296.87,301.30,1008.26,81.59
1,2011-01-01,4.37,300.22,297.02,299.73,1008.20,87.66
2,2011-01-02,50.81,298.95,297.32,298.57,1009.91,89.85
3,2011-01-03,5.23,299.15,296.96,298.76,1011.01,85.68
4,2011-01-04,0.05,300.15,296.64,299.64,1010.78,81.46


In [3]:
df_cases = pd.read_csv('./data/dengue/hdx/doh-epi-dengue-cases-iloilo.csv')
df_cases.head()

,loc,cases,deaths,date,Region,year
0,ILOILO CITY,19,0.0,10/01/2016,REGION VI-WESTERN VISAYAS,2016
1,ILOILO CITY,23,0.0,17/01/2016,REGION VI-WESTERN VISAYAS,2016
2,ILOILO CITY,31,0.0,24/01/2016,REGION VI-WESTERN VISAYAS,2016
3,ILOILO CITY,26,0.0,31/01/2016,REGION VI-WESTERN VISAYAS,2016
4,ILOILO CITY,10,1.0,07/02/2016,REGION VI-WESTERN VISAYAS,2016


## Merging the two datasets
Here, since we have few data points for 2021, we will remove it

In [4]:
# Fixing the format
df_cases['date'] = pd.to_datetime(df_cases['date'], format='%d/%m/%Y')
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y-%m-%d')

In [5]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   loc     258 non-null    object        
 1   cases   258 non-null    int64         
 2   deaths  258 non-null    float64       
 3   date    258 non-null    datetime64[ns]
 4   Region  258 non-null    object        
 5   year    258 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 12.2+ KB


In [6]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           5061 non-null   datetime64[ns]
 1   precipitation  5061 non-null   float64       
 2   max_temp       5061 non-null   float64       
 3   min_temp       5061 non-null   float64       
 4   temperature    5061 non-null   float64       
 5   pressure       5061 non-null   float64       
 6   humidity       5061 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 276.9 KB


In [7]:
# include only 2016 to 2020
df_cases = df_cases[df_cases['year'] <= 2020]
df_weather = df_weather[(df_weather['date'].dt.year <= 2020) & (df_weather['date'].dt.year >=2016)]


In [8]:
# Count weeks in df_cases
week_column = []
for i in range(len(df_cases)):
    week_column.append(str(df_cases.iloc[i]['date'].year) + '-' + str(i%52+1))
df_cases['week'] = week_column
df_cases

,loc,cases,deaths,date,Region,year,week
0,ILOILO CITY,19,0.0,2016-01-10,REGION VI-WESTERN VISAYAS,2016,2016-1
1,ILOILO CITY,23,0.0,2016-01-17,REGION VI-WESTERN VISAYAS,2016,2016-2
2,ILOILO CITY,31,0.0,2016-01-24,REGION VI-WESTERN VISAYAS,2016,2016-3
3,ILOILO CITY,26,0.0,2016-01-31,REGION VI-WESTERN VISAYAS,2016,2016-4
4,ILOILO CITY,10,1.0,2016-02-07,REGION VI-WESTERN VISAYAS,2016,2016-5
...,...,...,...,...,...,...,...
253,ILOILO CITY,8,0.0,2020-11-29,REGION VI-WESTERN VISAYAS,2020,2020-46
254,ILOILO CITY,1,0.0,2020-12-06,REGION VI-WESTERN VISAYAS,2020,2020-47
255,ILOILO CITY,0,0.0,2020-12-13,REGION VI-WESTERN VISAYAS,2020,2020-48
256,ILOILO CITY,3,0.0,2020-12-20,REGION VI-WESTERN VISAYAS,2020,2020-49


In [9]:
# for merging, use pandas mergeasof
df_merged = pd.merge_asof(df_weather.sort_values('date'), df_cases.sort_values('date'), on='date')
unnecessary_columns = ['max_temp', 'min_temp','loc','deaths', 'Region', 'year']
df_merged.drop(columns=unnecessary_columns, inplace=True)
df_merged.dropna(inplace=True)
df_merged = df_merged[["week","date","cases","temperature","pressure", "humidity", "precipitation"]]
df_merged.head()

,week,date,cases,temperature,pressure,humidity,precipitation
9,2016-1,2016-01-10,19.0,302.75,1014.0,72.0,0.00
10,2016-1,2016-01-11,19.0,304.20,1012.0,73.0,0.00
11,2016-1,2016-01-12,19.0,303.51,1013.0,75.0,0.61
12,2016-1,2016-01-13,19.0,302.85,1012.0,76.0,0.41
13,2016-1,2016-01-14,19.0,303.90,1013.0,64.0,0.00


## Making a weekly dataset
To get the granularity of the cases, we will need aggregate the other columns

In [10]:
df_merged_weekly_2016_2020 = df_merged.groupby('week').agg({"cases":"mean","temperature":"mean","pressure":"mean", "humidity":"mean", "precipitation":"sum"})
for_dt = df_merged
for_dt.drop_duplicates(subset='week', inplace=True)
for_dt = for_dt[['week','date']]
df_merged_weekly_2016_2020.reset_index('week', inplace=True)
df_merged_weekly_2016_2020 = pd.merge(for_dt, df_merged_weekly_2016_2020, on='week')
df_merged_weekly_2016_2020.sort_values('date', inplace=True)
df_merged_weekly_2016_2020.head()

,week,date,cases,temperature,pressure,humidity,precipitation
0,2016-1,2016-01-10,19.0,303.437143,1012.285714,71.428571,1.02
1,2016-2,2016-01-17,23.0,303.908571,1012.857143,65.571429,0.00
2,2016-3,2016-01-24,31.0,302.251429,1014.428571,76.857143,32.85
3,2016-4,2016-01-31,26.0,302.778571,1011.142857,75.142857,17.05
4,2016-5,2016-02-07,10.0,303.285714,1012.428571,70.571429,3.70


## Merging with 2022 to 2024
We'll use the other dataset that we have to merge them together

In [11]:
df_merged_weekly_2022_2024 = pd.read_csv('df_merged_weekly.csv', index_col=0)
df_merged_weekly_2022_2024.head()

,week,dt,Cases,temp,feels_like,pressure,humidity,precipitation
0,2022-1,2022-01-01,7.0,298.38,299.64,1013.0,87.0,3.14
1,2022-2,2022-01-08,1.0,302.23,308.88,1011.0,68.0,0.00
2,2022-3,2022-01-15,2.0,300.74,304.09,1013.0,58.0,0.00
3,2022-4,2022-01-22,3.0,302.42,309.89,1010.0,63.0,0.00
4,2022-5,2022-01-29,3.0,303.37,308.69,1010.0,64.0,0.42


In [12]:
# Rename the columns and drop columns that is not present in the previous dataset that we worked with
df_merged_weekly_2022_2024.rename(columns={"dt":"date", "Cases": "cases", "temp":"temperature"}, inplace=True)
df_merged_weekly_2022_2024.drop(columns=["feels_like"], inplace=True)
df_merged_weekly_2022_2024['date'] = pd.to_datetime(df_merged_weekly_2022_2024['date'])
df_merged_weekly_2022_2024.head()

,week,date,cases,temperature,pressure,humidity,precipitation
0,2022-1,2022-01-01,7.0,298.38,1013.0,87.0,3.14
1,2022-2,2022-01-08,1.0,302.23,1011.0,68.0,0.00
2,2022-3,2022-01-15,2.0,300.74,1013.0,58.0,0.00
3,2022-4,2022-01-22,3.0,302.42,1010.0,63.0,0.00
4,2022-5,2022-01-29,3.0,303.37,1010.0,64.0,0.42


In [13]:
df_all = pd.concat([df_merged_weekly_2016_2020,df_merged_weekly_2022_2024])
df_all.head()

,week,date,cases,temperature,pressure,humidity,precipitation
0,2016-1,2016-01-10,19.0,303.437143,1012.285714,71.428571,1.02
1,2016-2,2016-01-17,23.0,303.908571,1012.857143,65.571429,0.00
2,2016-3,2016-01-24,31.0,302.251429,1014.428571,76.857143,32.85
3,2016-4,2016-01-31,26.0,302.778571,1011.142857,75.142857,17.05
4,2016-5,2016-02-07,10.0,303.285714,1012.428571,70.571429,3.70


In [14]:
# save as csv
df_all.reset_index(inplace=True)
df_all.to_csv('df_merged_weekly_all.csv')